In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
import pickle

In [2]:
dfm= pd.read_csv('spam_ham_dataset.csv')
dfm.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [3]:
dfm.shape

(5171, 4)

In [4]:
dfm.columns

Index(['Unnamed: 0', 'label', 'text', 'label_num'], dtype='object')

In [5]:
dfm.drop_duplicates(inplace=True)

In [6]:
dfm.shape

(5171, 4)

In [7]:
dfm.isnull().sum()

Unnamed: 0    0
label         0
text          0
label_num     0
dtype: int64

In [8]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\welcome\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
def procees_val(text):
    non_pun_words=[char for char in text if char not in string.punctuation]
    non_pun_words="".join(non_pun_words)
    clean_words=[word for word in non_pun_words.split() if word.lower() not in stopwords.words("english")]
    return clean_words



In [10]:
dfm['text'].head().apply(procees_val)

0    [Subject, enron, methanol, meter, 988291, foll...
1    [Subject, hpl, nom, january, 9, 2001, see, att...
2    [Subject, neon, retreat, ho, ho, ho, around, w...
3    [Subject, photoshop, windows, office, cheap, m...
4    [Subject, indian, springs, deal, book, teco, p...
Name: text, dtype: object

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
model=CountVectorizer(analyzer=procees_val)
coumat=model.fit_transform(dfm["text"])

In [13]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(coumat, dfm['label_num'], test_size=0.20, random_state=0)
coumat.shape

(5171, 50381)

In [14]:
from sklearn.naive_bayes import MultinomialNB


In [15]:
clf=MultinomialNB()
clf.fit(xtrain,ytrain)


MultinomialNB()

In [16]:
print(clf.predict(xtrain))
print(ytrain.values)

[0 0 0 ... 1 0 0]
[0 0 0 ... 1 0 0]


In [17]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [18]:
mat=clf.predict(xtrain)
print(classification_report(ytrain,mat))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2940
           1       0.98      0.97      0.98      1196

    accuracy                           0.99      4136
   macro avg       0.99      0.98      0.98      4136
weighted avg       0.99      0.99      0.99      4136



In [19]:
print("confusion Matrix :",confusion_matrix(ytrain,mat))

confusion Matrix : [[2918   22]
 [  30 1166]]


In [20]:
print(accuracy_score(ytrain,mat))

0.9874274661508704


In [21]:
from joblib import dump,load
dump(clf,'spam.joblib')

['spam.joblib']

In [22]:
print(clf.predict(xtest))
print(ytest.values)

[0 0 0 ... 0 1 0]
[0 0 0 ... 0 1 0]


In [23]:
mat=clf.predict(xtest)
print(classification_report(ytest,mat))
print("confusion Matrix :",confusion_matrix(ytest,mat))
print(accuracy_score(ytest,mat))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       732
           1       0.95      0.96      0.96       303

    accuracy                           0.97      1035
   macro avg       0.97      0.97      0.97      1035
weighted avg       0.97      0.97      0.97      1035

confusion Matrix : [[718  14]
 [ 13 290]]
0.9739130434782609


In [24]:
pickle.dump(clf,open('model.pkl','wb'))



In [25]:
model=pickle.load(open("model.pkl",'rb'))

In [30]:
xtest.shape



(1035, 50381)